**Forked from dune_dweller's excellent notebook. I try using xgboost gblinear instead of logistic regression but get slightly worse results.

This notebook shows how to build a linear model on features from apps, app labels, phone brands and device models. It uses LogisticRegression classifier from sklearn. 

It also shows an efficient way of constructing bag-of-apps and bag-of-labels features without concatenating a bunch of strings.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss

## Load data

In [ ]:
datadir = '../input'
gatrain = pd.read_csv(os.path.join(datadir,'gender_age_train.csv'),
                      index_col='device_id')
gatest = pd.read_csv(os.path.join(datadir,'gender_age_test.csv'),
                     index_col = 'device_id')
phone = pd.read_csv(os.path.join(datadir,'phone_brand_device_model.csv'))
# Get rid of duplicate device ids in phone
phone = phone.drop_duplicates('device_id',keep='first').set_index('device_id')
events = pd.read_csv(os.path.join(datadir,'events.csv'),
                     parse_dates=['timestamp'], index_col='event_id')
appevents = pd.read_csv(os.path.join(datadir,'app_events.csv'), 
                        usecols=['event_id','app_id','is_active'],
                        dtype={'is_active':bool})
applabels = pd.read_csv(os.path.join(datadir,'app_labels.csv'))

## Feature engineering

The features I'm going to use include:

* phone brand
* device model
* installed apps
* app labels

I'm going to one-hot encode everything and sparse matrices will help deal with a very large number of features.

### Phone brand

As preparation I create two columns that show which train or test set row a particular device_id belongs to.

In [ ]:
gatrain['trainrow'] = np.arange(gatrain.shape[0])
gatest['testrow'] = np.arange(gatest.shape[0])

A sparse matrix of features can be constructed in various ways. I use this constructor:

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
    where ``data``, ``row_ind`` and ``col_ind`` satisfy the
    relationship ``a[row_ind[k], col_ind[k]] = data[k]``
    
It lets me specify which values to put into which places in a sparse matrix. For phone brand data the `data` array will be all ones, `row_ind` will be the row number of a device and `col_ind` will be the number of brand.

In [ ]:
brandencoder = LabelEncoder().fit(phone.phone_brand)
phone['brand'] = brandencoder.transform(phone['phone_brand'])
gatrain['brand'] = phone['brand']
gatest['brand'] = phone['brand']
Xtr_brand = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.brand)))
Xte_brand = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.brand)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_brand.shape, Xte_brand.shape))

### Device model

In [ ]:
m = phone.phone_brand.str.cat(phone.device_model)
modelencoder = LabelEncoder().fit(m)
phone['model'] = modelencoder.transform(m)
gatrain['model'] = phone['model']
gatest['model'] = phone['model']
Xtr_model = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.model)))
Xte_model = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.model)))
print('Model features: train shape {}, test shape {}'.format(Xtr_model.shape, Xte_model.shape))

### Installed apps features

For each device I want to mark which apps it has installed. So I'll have as many feature columns as there are distinct apps.

Apps are linked to devices through events. So I do the following:

- merge `device_id` column from `events` table to `app_events`
- group the resulting dataframe by `device_id` and `app` and aggregate
- merge in `trainrow` and `testrow` columns to know at which row to put each device in the features matrix

In [ ]:
appencoder = LabelEncoder().fit(appevents.app_id)
appevents['app'] = appencoder.transform(appevents.app_id)
napps = len(appencoder.classes_)
deviceapps = (appevents.merge(events[['device_id']], how='left',left_on='event_id',right_index=True)
                       .groupby(['device_id','app'])['app'].agg(['size'])
                       .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                       .merge(gatest[['testrow']], how='left', left_index=True, right_index=True)
                       .reset_index())
deviceapps.head()

Now I can build a feature matrix where the `data` is all ones, `row_ind` comes from `trainrow` or `testrow` and `col_ind` is the label-encoded `app_id`.

In [ ]:
d = deviceapps.dropna(subset=['trainrow'])
Xtr_app = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.app)), 
                      shape=(gatrain.shape[0],napps))
d = deviceapps.dropna(subset=['testrow'])
Xte_app = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.app)), 
                      shape=(gatest.shape[0],napps))
print('Apps data: train shape {}, test shape {}'.format(Xtr_app.shape, Xte_app.shape))

### App labels features

These are constructed in a way similar to apps features by merging `app_labels` with the `deviceapps` dataframe we created above.

In [ ]:
applabels = applabels.loc[applabels.app_id.isin(appevents.app_id.unique())]
applabels['app'] = appencoder.transform(applabels.app_id)
labelencoder = LabelEncoder().fit(applabels.label_id)
applabels['label'] = labelencoder.transform(applabels.label_id)
nlabels = len(labelencoder.classes_)

In [ ]:
devicelabels = (deviceapps[['device_id','app']]
                .merge(applabels[['app','label']])
                .groupby(['device_id','label'])['app'].agg(['size'])
                .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                .merge(gatest[['testrow']], how='left', left_index=True, right_index=True)
                .reset_index())
devicelabels.head()

In [ ]:
d = devicelabels.dropna(subset=['trainrow'])
Xtr_label = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.label)), 
                      shape=(gatrain.shape[0],nlabels))
d = devicelabels.dropna(subset=['testrow'])
Xte_label = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.label)), 
                      shape=(gatest.shape[0],nlabels))
print('Labels data: train shape {}, test shape {}'.format(Xtr_label.shape, Xte_label.shape))

### Concatenate all features

In [ ]:
Xtrain = hstack((Xtr_brand, Xtr_model, Xtr_app, Xtr_label), format='csr')
Xtest =  hstack((Xte_brand, Xte_model, Xte_app, Xte_label), format='csr')
print('All features: train shape {}, test shape {}'.format(Xtrain.shape, Xtest.shape))

In [ ]:
Xtrain

## Try xgboost:
dune_dweller's original script uses a regularized logistic regression to tackle the problem so I was curious how a linear boosted model would do. To my surprise I haven't been able to beat the score of one linear model (granted I haven't been spent too much time on the parameter tuning)

In [ ]:
targetencoder = LabelEncoder().fit(gatrain.group)
y = targetencoder.transform(gatrain.group)
nclasses = len(targetencoder.classes_)

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(Xtrain, y)

In [ ]:
params = {
        "eta": 0.1,
        "booster": "gblinear",
        "objective": "multi:softprob",
        "alpha": 4,
        "lambda": 0,
        "silent": 1,
        "seed": 1233,
        "num_class": 12,
        "eval_metric": "mlogloss"
    }

In [ ]:
xgb.cv(params, dtrain, 
       num_boost_round=50, 
       #early_stopping_rounds = 5, 
       maximize = False)

It seems we can only get around 2.279 using xgboost. 

## Try Random Forests:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
#model.fit(Xtrain, y) hmm this is going to take too long.

## Predict on test data

In [ ]:
model = xgb.train(params, dtrain, num_boost_round=25)

In [ ]:
dtest = xgb.DMatrix(Xtest)

In [ ]:
model.predict(dtest)
pred = pd.DataFrame(model.predict(dtest), index = gatest.index, columns=targetencoder.classes_)

In [ ]:
pred.head()

In [ ]:
pred.to_csv('xgb_subm.csv',index=True) 

The submission above scores 2.26735 on the LB, slightly worse than 	2.26508 for dune_dweller's original script. I guess this should a good lesson to not ignore logistic regression in the future :)